# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
%%capture
!pip install "dlt[duckdb]"

In [8]:
pip install duckdb

Note: you may need to restart the kernel to use updated packages.


In [12]:
import dlt, duckdb

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

# for sqrt_value in generator:
#     print(sqrt_value)

#Question 1: What is the sum of the outputs of the generator for limit = 5?

total = sum(generator)
print(total)

8.382332347441762


# Question 2: What is the 13th number yielded

In [9]:
limit = 13
generator_13 = square_root_generator(limit)
gen = list(generator_13)#this does not optimize memory and not ideal
print(f"The 13th number yielded is {gen[12]}")

The 13th number yielded is 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [23]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [13]:
my_pipeline = dlt.pipeline(destination='duckdb', dataset_name='peoples_dataset')

info = my_pipeline.run(people_1(),
										table_name="people_table",
										write_disposition="replace")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset peoples_dataset
The duckdb destination used duckdb:////Users/apple/Desktop/DE_Zoomcamp/Data-Engineering-Zoomcamp-2024/dlt_workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708287723.789469 is LOADED and contains no failed jobs


In [14]:
conn = duckdb.connect(f"{my_pipeline.pipeline_name}.duckdb")

In [18]:
conn.sql(f"SET search_path = '{my_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_table        │
│ people_table_merged │
└─────────────────────┘

In [25]:
people_1_df = conn.sql("SELECT * FROM peoples_dataset.people_table").df()
display(people_1_df)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708287723.789469,37YUrauOnYcONg,None
1,2,Person_2,27,City_A,1708287723.789469,P0LHyWt7aM8p/w,None
2,3,Person_3,28,City_A,1708287723.789469,YBwyClMRXqYhyw,None
3,4,Person_4,29,City_A,1708287723.789469,oaMCWdJ0JZ1Onw,None
4,5,Person_5,30,City_A,1708287723.789469,YVgVV2AYTr7wvg,None


# Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

In [26]:
total_ages = people_1_df['age'].sum()
print(total_ages)

140


# 2.Append the second generator to the same table as the first.

In [27]:
info = my_pipeline.run(people_2(),
										table_name="people_table",
										write_disposition="append")

In [28]:
people_2_df = conn.sql("SELECT * FROM peoples_dataset.people_table").df()
display(people_2_df)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708287723.789469,37YUrauOnYcONg,None
1,2,Person_2,27,City_A,1708287723.789469,P0LHyWt7aM8p/w,None
2,3,Person_3,28,City_A,1708287723.789469,YBwyClMRXqYhyw,None
3,4,Person_4,29,City_A,1708287723.789469,oaMCWdJ0JZ1Onw,None
4,5,Person_5,30,City_A,1708287723.789469,YVgVV2AYTr7wvg,None
5,3,Person_3,33,City_B,1708288018.621891,sdn/7rvlbIS74Q,Job_3
6,4,Person_4,34,City_B,1708288018.621891,kBS18M6eXzvfHQ,Job_4
7,5,Person_5,35,City_B,1708288018.621891,wfj1o4w1uy3avg,Job_5
8,6,Person_6,36,City_B,1708288018.621891,5KvCGb1ICAukGA,Job_6
9,7,Person_7,37,City_B,1708288018.621891,Cuu37/LitAYYVg,Job_7


# 3.After correctly appending the data, calculate the sum of all ages of people.

In [29]:
total_ages = people_2_df['age'].sum()
print(total_ages)

353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [30]:
info = my_pipeline.run(people_1(),
										table_name="people_table_merged",
										write_disposition="replace",primary_key="id")


In [32]:
merged_df = conn.sql("SELECT * FROM peoples_dataset.people_table_merged").df()
display(merged_df)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708288187.720084,XRSmDU48AuEtGA,None
1,2,Person_2,27,City_A,1708288187.720084,df6vcpI6SX5HfQ,None
2,3,Person_3,28,City_A,1708288187.720084,AkWyqg9BwFDwIw,None
3,4,Person_4,29,City_A,1708288187.720084,ySc99AHyB+nL6Q,None
4,5,Person_5,30,City_A,1708288187.720084,RaTQBweZF2TYnA,None


In [33]:
info = my_pipeline.run(people_2(),
										table_name="people_table_merged",
										write_disposition="merge",primary_key="id")


In [34]:
merged_df = conn.sql("SELECT * FROM peoples_dataset.people_table_merged").df()
display(merged_df)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708288187.720084,XRSmDU48AuEtGA,None
1,2,Person_2,27,City_A,1708288187.720084,df6vcpI6SX5HfQ,None
2,4,Person_4,34,City_B,1708288200.72125,3z6/ZgFaA6kt0Q,Job_4
3,5,Person_5,35,City_B,1708288200.72125,6eZCiCim8Jewhw,Job_5
4,3,Person_3,33,City_B,1708288200.72125,HIvTeK8vOlBgJg,Job_3
5,6,Person_6,36,City_B,1708288200.72125,q6l/hGDOIwxpRQ,Job_6
6,7,Person_7,37,City_B,1708288200.72125,jun/x3A1FVk+xA,Job_7
7,8,Person_8,38,City_B,1708288200.72125,ISAAz7wR7Afs4A,Job_8


# Question: Calculate the sum of ages of all the people loaded as described above.

In [35]:
total_ages = merged_df['age'].sum()
print(total_ages)

266


# Solution: First make sure that the following modules are installed:

In [21]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

UsageError: Line magic function `%%capture` not found.


In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX